In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
path = "mnist.npz"

In [13]:
def train_mnist_conv():
    class mycallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy')>0.998 and logs.get('accuracy')is not None):
                print("\nReached 99.8% accuracy so cancelling training!")
                self.model.stop_training= True

    mnist = tf.keras.datasets.mnist
    (training_images, training_labels), (test_images, test_labels) = mnist.load_data(path=path)

    callbacks=mycallback()
    training_images=training_images.reshape(60000, 28, 28, 1)
    training_images=training_images / 255.0
    test_images = test_images.reshape(10000, 28, 28, 1)
    test_images=test_images/255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
    return history.epoch, history.history['accuracy'][-1]

In [14]:
train_mnist_conv()


2021-11-11 17:32:26.617229: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-11 17:32:27.069170: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2947 - accuracy: 0.9140

([0, 1, 2, 3, 4, 5, 6, 7, 8], 0.998116672039032)